<h1 style="color: #ffffcc;">Import Modules</h1>

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
import os
import sys

In [16]:
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))
sys.path.append(os.path.abspath('../scripts'))

In [17]:
from scripts.Data_loader import load_data
from scripts.analysis import feature_engineering, handle_missing_values, encode_categorical_data, scale_data

In [18]:
Data = load_data(r'C:\Users\fikad\Desktop\10acedamy\INSURANCE_ANALYSIS_\Data\MachineLearningRating_v3.csv')
Data.head(5)

c:\Users\fikad\Desktop\10acedamy\INSURANCE_ANALYSIS_\scripts\Data_loader.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


<h2 style="color: #ffffbb;">Data Preparation</h2>

In [19]:
Data.shape

(1000098, 52)

In [20]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000098 entries, 0 to 1000097
Data columns (total 52 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000098 non-null  int64  
 1   PolicyID                  1000098 non-null  int64  
 2   TransactionMonth          1000098 non-null  object 
 3   IsVATRegistered           1000098 non-null  bool   
 4   Citizenship               1000098 non-null  object 
 5   LegalType                 1000098 non-null  object 
 6   Title                     1000098 non-null  object 
 7   Language                  1000098 non-null  object 
 8   Bank                      854137 non-null   object 
 9   AccountType               959866 non-null   object 
 10  MaritalStatus             991839 non-null   object 
 11  Gender                    990562 non-null   object 
 12  Country                   1000098 non-null  object 
 13  Province                  1

In [21]:
# Identify numerical and categorical columns
numerical_columns = Data.select_dtypes(include=['number']).columns.tolist()
categorical_columns = Data.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numerical columns:", numerical_columns)
print("Categorical columns:", categorical_columns)

Numerical columns: ['UnderwrittenCoverID', 'PolicyID', 'PostalCode', 'mmcode', 'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CustomValueEstimate', 'NumberOfVehiclesInFleet', 'SumInsured', 'CalculatedPremiumPerTerm', 'TotalPremium', 'TotalClaims']
Categorical columns: ['TransactionMonth', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'VehicleType', 'make', 'Model', 'bodytype', 'VehicleIntroDate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'TermFrequency', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType']


<h3 style="color: #ffffaa;">Handling missing value</h3>

In [22]:
# Step 1: Check missing values percentage
missing_percentage = Data.isnull().mean() * 100
print(missing_percentage)

UnderwrittenCoverID           0.000000
PolicyID                      0.000000
TransactionMonth              0.000000
IsVATRegistered               0.000000
Citizenship                   0.000000
LegalType                     0.000000
Title                         0.000000
Language                      0.000000
Bank                         14.594670
AccountType                   4.022806
MaritalStatus                 0.825819
Gender                        0.953507
Country                       0.000000
Province                      0.000000
PostalCode                    0.000000
MainCrestaZone                0.000000
SubCrestaZone                 0.000000
ItemType                      0.000000
mmcode                        0.055195
VehicleType                   0.055195
RegistrationYear              0.000000
make                          0.055195
Model                         0.055195
Cylinders                     0.055195
cubiccapacity                 0.055195
kilowatts                

In [23]:
print("Before Handling Missing Values:")
print(Data.head)

# Handle missing values
processed_data = handle_missing_values(Data)

print("\nAfter Handling Missing Values:")
print(processed_data.head)

Before Handling Missing Values:
<bound method NDFrame.head of          UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0                     145249     12827  2015-03-01 00:00:00             True   
1                     145249     12827  2015-05-01 00:00:00             True   
2                     145249     12827  2015-07-01 00:00:00             True   
3                     145255     12827  2015-05-01 00:00:00             True   
4                     145255     12827  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:0

c:\Users\fikad\Desktop\10acedamy\INSURANCE_ANALYSIS_\scripts\analysis.py:68: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mean(), inplace=True)
c:\Users\fikad\Desktop\10acedamy\INSURANCE_ANALYSIS_\scripts\analysis.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b


After Handling Missing Values:
<bound method NDFrame.head of          UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0                     145249     12827  2015-03-01 00:00:00             True   
1                     145249     12827  2015-05-01 00:00:00             True   
2                     145249     12827  2015-07-01 00:00:00             True   
3                     145255     12827  2015-05-01 00:00:00             True   
4                     145255     12827  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:0

<h3 style="color: #ffffaa;">Feature Engineering</h3>


In [24]:
# Apply the feature engineering method
engineered_data = feature_engineering(processed_data)

# Display the result
print(engineered_data.head)

c:\Users\fikad\Desktop\10acedamy\INSURANCE_ANALYSIS_\scripts\analysis.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['VehicleIntroDate'] = pd.to_datetime(data['VehicleIntroDate'], errors='coerce')


<bound method NDFrame.head of          UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0                     145249     12827  2015-03-01 00:00:00             True   
1                     145249     12827  2015-05-01 00:00:00             True   
2                     145249     12827  2015-07-01 00:00:00             True   
3                     145255     12827  2015-05-01 00:00:00             True   
4                     145255     12827  2015-07-01 00:00:00             True   
...                      ...       ...                  ...              ...   
1000093                31520       389  2015-04-01 00:00:00            False   
1000094                31520       389  2015-06-01 00:00:00            False   
1000095                31520       389  2015-08-01 00:00:00            False   
1000096                31519       389  2014-07-01 00:00:00            False   
1000097                31519       389  2015-02-01 00:00:00            False   

        C

In [25]:
#  Select relevant features for insurance premium prediction
relevant_features = [
    'IsVATRegistered', 'LegalType', 'MaritalStatus', 'Gender',
    'VehicleType', 'RegistrationYear', 'make', 'Model','PostalCode','Province',
    'bodytype','SumInsured', 'CalculatedPremiumPerTerm',
    'CoverCategory', 'CoverType', 'CoverGroup',
    'ClaimsToPremiumRatio', 'VehicleAge', 'IsNewVehicle','TotalClaims','TotalPremium'
]
selected_features = [feature for feature in relevant_features if feature in engineered_data.columns]
selected_data = Data[selected_features]

In [26]:
selected_data.head()

,IsVATRegistered,LegalType,MaritalStatus,Gender,VehicleType,RegistrationYear,make,Model,PostalCode,Province,...,SumInsured,CalculatedPremiumPerTerm,CoverCategory,CoverType,CoverGroup,ClaimsToPremiumRatio,VehicleAge,IsNewVehicle,TotalClaims,TotalPremium
0,True,Close Corporation,Not specified,Not specified,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,1459,Gauteng,...,0.01,25.0000,Windscreen,Windscreen,Comprehensive - Taxi,0.0,22,1,0.0,21.929825
1,True,Close Corporation,Not specified,Not specified,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,1459,Gauteng,...,0.01,25.0000,Windscreen,Windscreen,Comprehensive - Taxi,0.0,22,1,0.0,21.929825
2,True,Close Corporation,Not specified,Not specified,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,1459,Gauteng,...,0.01,25.0000,Windscreen,Windscreen,Comprehensive - Taxi,0.0,22,1,0.0,0.000000
3,True,Close Corporation,Not specified,Not specified,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,1459,Gauteng,...,119300.00,584.6468,Own damage,Own Damage,Comprehensive - Taxi,0.0,22,1,0.0,512.848070
4,True,Close Corporation,Not specified,Not specified,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,1459,Gauteng,...,119300.00,584.6468,Own damage,Own Damage,Comprehensive - Taxi,0.0,22,1,0.0,0.000000


In [27]:
selected_data.select_dtypes(include=['object','bool']).columns

Index(['IsVATRegistered', 'LegalType', 'MaritalStatus', 'Gender',
       'VehicleType', 'make', 'Model', 'Province', 'bodytype', 'CoverCategory',
       'CoverType', 'CoverGroup'],
      dtype='object')

<h3 style="color: #ffffaa;">Encoding Categorical Data</h3>

In [28]:
print("Before Encoding:")
print(selected_data)

# One-Hot Encoding
onehot_encoded_data = encode_categorical_data(selected_data, method='onehot', columns=['IsVATRegistered', 'LegalType', 'MaritalStatus', 'Gender', 
                      'VehicleType', 'make', 'Model', 'bodytype', 'Province',
                      'CoverCategory', 'CoverType', 'CoverGroup'])

print("\nAfter One-Hot Encoding:")
print(onehot_encoded_data)

# Label Encoding
label_encoded_data = encode_categorical_data(selected_data, method='label', columns=['IsVATRegistered', 'LegalType', 'MaritalStatus', 'Gender', 
                      'VehicleType', 'make', 'Model', 'bodytype', 'Province',
                      'CoverCategory', 'CoverType', 'CoverGroup'])
print("\nAfter Label Encoding:")
print(label_encoded_data)

Before Encoding:
         IsVATRegistered          LegalType  MaritalStatus         Gender  \
0                   True  Close Corporation  Not specified  Not specified   
1                   True  Close Corporation  Not specified  Not specified   
2                   True  Close Corporation  Not specified  Not specified   
3                   True  Close Corporation  Not specified  Not specified   
4                   True  Close Corporation  Not specified  Not specified   
...                  ...                ...            ...            ...   
1000093            False         Individual         Single           Male   
1000094            False         Individual         Single           Male   
1000095            False         Individual         Single           Male   
1000096            False         Individual         Single           Male   
1000097            False         Individual         Single           Male   

               VehicleType  RegistrationYear           mak


<h3 style="color: #ffffaa;">Scaling</h3>

In [29]:
print("Before Scaling:")
print(label_encoded_data)

# Apply standard scaling
scaled_data_standard = scale_data(label_encoded_data, method='standard')
print("\nAfter Standard Scaling:")
print(scaled_data_standard)

Before Scaling:
         IsVATRegistered  LegalType  MaritalStatus  Gender  VehicleType  \
0                      1          0              1       2            4   
1                      1          0              1       2            4   
2                      1          0              1       2            4   
3                      1          0              1       2            4   
4                      1          0              1       2            4   
...                  ...        ...            ...     ...          ...   
1000093                0          1              2       1            4   
1000094                0          1              2       1            4   
1000095                0          1              2       1            4   
1000096                0          1              2       1            4   
1000097                0          1              2       1            4   

         RegistrationYear  make  Model  PostalCode  Province  ...  SumInsured  \
0 


<h3 style="color: #ffffaa;">Train-Test Split</h3>

In [30]:
# Split data into X and y
X=scaled_data_standard.drop(['TotalPremium','TotalClaims'],axis=1)
y=scaled_data_standard['TotalPremium']

In [31]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Display the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (700068, 19)
X_test shape: (300030, 19)
y_train shape: (700068,)
y_test shape: (300030,)



<h1 style="color: #ffffaa;">Modeling Techniques
</h1>

In [ ]:
# Import necessary libraries
import shap
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Step 1: Train the Linear Regression model (same as above)
model = LinearRegression()
model.fit(X_train, y_train)

# Step 2: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 3: Evaluate the model with metrics
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
r2 = r2_score(y_test, y_pred)  # R-squared value

# Print evaluation results
print("Linear Regression Performance:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

# Step 4: Feature Importance Analysis (Linear regression coefficients)
print("\nModel Coefficients (Feature Importance):")
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': model.coef_})
coefficients['Importance'] = coefficients['Coefficient'].abs()  # Importance as absolute value of coefficient
coefficients = coefficients.sort_values(by='Importance', ascending=False)
print(coefficients)

# Step 5: SHAP (SHapley Additive exPlanations) for feature importance analysis
# Step 5: SHAP (SHapley Additive exPlanations) for feature importance analysis
explainer = shap.Explainer(model, X_train)  # Create SHAP explainer
shap_values = explainer(X_test)  # Get SHAP values for test data

# Plot SHAP feature importance
shap.summary_plot(shap_values, X_test)


c:\Users\fikad\Desktop\10acedamy\INSURANCE_ANALYSIS_\Week-3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Linear Regression Performance:
Mean Squared Error (MSE): 0.5409982163566123
Mean Absolute Error (MAE): 0.2598312858487649
R-squared (R2): 0.4590461001367405

Model Coefficients (Feature Importance):
                     Feature  Coefficient  Importance
12  CalculatedPremiumPerTerm     0.589845    0.589845
14                 CoverType     0.570592    0.570592
13             CoverCategory    -0.542325    0.542325
15                CoverGroup    -0.030202    0.030202
5           RegistrationYear     0.023537    0.023537
9                   Province    -0.016408    0.016408
11                SumInsured    -0.013549    0.013549
1                  LegalType     0.012279    0.012279
10                  bodytype    -0.011987    0.011987
3                     Gender     0.011823    0.011823
7                      Model     0.010377    0.010377
4                VehicleType    -0.009324    0.009324
2              MaritalStatus    -0.009287    0.009287
8                 PostalCode     0.008823    

In [ ]:
# Import necessary libraries
import shap
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Step 1: Train the Decision Tree model
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)  

# Step 2: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 3: Evaluate the model with metrics
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
r2 = r2_score(y_test, y_pred)  # R-squared value

# Print evaluation results
print("Decision Tree Performance:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

# Step 4: Feature Importance Analysis (using Decision Tree feature importances)
print("\nModel Feature Importance (Decision Tree):")
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

# Step 5: SHAP (SHapley Additive exPlanations) for feature importance analysis
explainer = shap.Explainer(model, X_train)  # Create SHAP explainer
shap_values = explainer(X_test)  # Get SHAP values for test data

# Plot SHAP feature importance
shap.summary_plot(shap_values, X_test)



In [ ]:
# Import necessary libraries
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Step 1: Train the Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)  

# Step 2: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 3: Evaluate the model with metrics
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
r2 = r2_score(y_test, y_pred)  # R-squared value

# Print evaluation results
print("Random Forest Performance:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

# Step 4: Feature Importance Analysis (using Random Forest feature importances)
print("\nModel Feature Importance (Random Forest):")
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

# Step 5: SHAP (SHapley Additive exPlanations) for feature importance analysis
explainer = shap.Explainer(model, X_train)  # Create SHAP explainer
shap_values = explainer(X_test)  # Get SHAP values for test data

# Plot SHAP feature importance
shap.summary_plot(shap_values, X_test)


In [ ]:
# Import necessary libraries
import shap
import matplotlib.pyplot as plt
from xgboost import XGBRegressor  # Import XGBoost Regressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Step 1: Train the XGBoost model
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)  

# Step 2: Make predictions on the test set
y_pred = model.predict(X_test)

# Step 3: Evaluate the model with metrics
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
r2 = r2_score(y_test, y_pred)  # R-squared value

# Print evaluation results
print("XGBoost Performance:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2):", r2)

# Step 4: Feature Importance Analysis (using XGBoost feature importances)
print("\nModel Feature Importance (XGBoost):")
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
print(feature_importance)

# Step 5: SHAP (SHapley Additive exPlanations) for feature importance analysis
explainer = shap.Explainer(model, X_train)  # Create SHAP explainer
shap_values = explainer(X_test)  # Get SHAP values for test data

# Plot SHAP feature importance
shap.summary_plot(shap_values, X_test)